In [ ]:
from __future__ import division
import math
import time
import scipy.special
import scipy.optimize
import matplotlib.pyplot as plt
import numpy as np
import pickle
import multiprocessing as mp
from functools import partial
np.random.seed(0)
def get_sigmaset(longer=False):
    '''
    longer: adds 4 more points in the range 30-70
    '''
    a = np.linspace(0,.25,num=6)
    a[0] = 0.5*a[1]
    b = np.linspace(.5,5,num=7)
    c = np.linspace(9,30,num=7)
    if longer==False:
        return np.hstack((np.hstack((a,b)),c))
    else:
        d = np.linspace(40,70,num=4)
        return np.hstack((np.hstack((a,b)),np.hstack((c,d))))

In [ ]:
#Settings
rev_curves = {'no','full','2stg'}
nsampling1stage = 100
muz    = 10
sigmaz = 1 # 1 # 4
sigmaset = get_sigmaset()
cost   = 0 # 0 # 3
pset   = np.linspace(cost,cost+15,num=200)

In [ ]:
def Fullinfo_given_mu(mu,sigma,px):
    def fbar(x):
        return -x*(scipy.special.erfc((x-mu)/(math.sqrt(2)*sigma))/2)
    def f(x):
        return -fbar(x)-px
    res = scipy.optimize.minimize(fbar,0)
    #print([res.x[0],f(res.x[0])])
    if f(res.x[0])>=0:
        fullinfo_threshold_temp = scipy.optimize.fsolve(f, 0)[0]
        fullinfo_adoption_temp = scipy.special.erfc((fullinfo_threshold_temp-mu)/(math.sqrt(2)*sigma))/2
    else:
        fullinfo_threshold_temp = np.inf
        fullinfo_adoption_temp = 0

    return (fullinfo_threshold_temp, fullinfo_adoption_temp)

def Fullinfo(sigmaz,muz,sigma,px,cost,nsampling1stage,stdnormal_presampled):
    adoptemp = np.zeros(nsampling1stage)
    thresholdtemp = np.zeros(nsampling1stage)
    for s in range(nsampling1stage):
        mu = muz + stdnormal_presampled[s]*sigmaz
        fullinfo_threshold_temp,fullinfo_adoption_temp = Fullinfo_given_mu(mu,sigma,px)
        adoptemp[s] = fullinfo_adoption_temp
        thresholdtemp[s] = fullinfo_threshold_temp
    fullinfo_rev=(px-cost)*adoptemp.mean()
    #print("px: {:.2f}, rev: {:.2f}".format(px,fullinfo_rev))
    return fullinfo_rev,thresholdtemp

In [ ]:
def Noinfo(sigmaz,muz,sigma,px,cost=0):
    sigmatemp = math.sqrt(((sigma**2)*(sigmaz**2))/(sigma**2+sigmaz**2)+sigma**2)
    def mutemp(x):
        return (x*sigmaz**2 + muz*sigma**2)/(sigmaz**2 + sigma**2)
    def fbar(x):
        return -x*(scipy.special.erfc((x-mutemp(x))/(math.sqrt(2)*sigmatemp))/2)
    def f(x):
        return -fbar(x)-px
    res = scipy.optimize.minimize(fbar,0)
    if f(res.x[0])>= 0:
        noinfo_threshold = scipy.optimize.fsolve(f, 0)[0]
        noinfo_adoption = scipy.special.erfc((noinfo_threshold-muz)/(math.sqrt(2)*math.sqrt(sigma**2+sigmaz**2)))/2
    else:
        noinfo_threshold=np.inf
        noinfo_adoption = 0
    noinfo_rev=(px-cost)*noinfo_adoption
    
    # print("px: {:.2f}, rev: {:.2f}, thres: {:.2f}, adop: {:.2f}, ".format(px,noinfo_rev,noinfo_threshold,noinfo_adoption))
    return noinfo_rev,noinfo_threshold,noinfo_adoption

In [ ]:
def Vstage(sigmaz,muz,sigma,p1,p2,v1set,nsampling2stage,stdnormal_presampled):
    
    def diff2(sigmaz,muz,sigma,p1,p2,v1,nsampling2stage,stdnormal_presampled):
        difftemp = np.zeros(nsampling2stage)
        LHT = np.zeros(nsampling2stage)
        
        mutemp = (v1*sigmaz**2 + muz*sigma**2)/(sigmaz**2 + sigma**2)
        sigmatemp = math.sqrt(((sigma**2)*(sigmaz**2))/(sigma**2+sigmaz**2))
        
        for s in range(nsampling2stage):
            mu = mutemp + stdnormal_presampled[s]*sigmatemp
            vfull,_= Fullinfo_given_mu(mu,sigma,p2)
            
            adopLHtemp=scipy.special.erfc((v1-mu)/(math.sqrt(2)*sigma))/2
            if v1 <= vfull: 
                LH= v1*adopLHtemp-p2
            else:
                LH = 0
            difftemp[s] = LH-(p1-p2)
            LHT[s] = LH
 
        diff=difftemp.mean()
        EL =-LHT.mean()
        return [diff,EL]
    
    diffv=[]
    absdiffv=[]
    if len(v1set)==0: #hackinsh code
        return np.inf,np.inf
    for v1 in v1set:
        error=diff2(sigmaz,muz,sigma,p1,p2,v1,nsampling2stage,stdnormal_presampled)[0]
        diffv.append(error)
        absdiffv.append(abs(error))
    if len(diffv)==0: #hackinsh code
        return np.inf,np.inf
    if max(diffv) < 0:
        return np.inf,max(diffv)
    #if min(diffv) >= 0:
    #    v1index=absdiffv.index(min(absdiffv))
    #    v1opt=v1set[v1index]
    #    return v1opt,min(diffv)
    #if min(absdiffv) > 1:
    #    return np.inf,max(diffv)
    
        
    val = min([i for i in diffv if i>=0])
    v1index = diffv.index(val)
    v1opt=v1set[v1index]
    return v1opt,diffv

In [ ]:
def Rev2stage(sigmaz,muz,sigma,p1,p2,v1set,cost,nsampling2stage,stdnormal_presampled):
    
    #opt value of v1
    v1,errorval= Vstage(sigmaz,muz,sigma,p1,p2,v1set,nsampling2stage,stdnormal_presampled)
    #print('v1',v1,'errorval for first stage constraint',errorval,'p1',p1,'p2',p2)

    revsamples = np.zeros(nsampling1stage)
    v2thresholds = np.zeros(nsampling1stage)
    firststage = (p1-p2)*(scipy.special.erfc((v1-muz)/(math.sqrt(2)*math.sqrt(sigmaz**2+sigma**2)))/2)
    #if np.isinf(v1):
    #    return 0,revsamples
    for s in range(nsampling2stage):
        mu = muz + stdnormal_presampled[s]*sigmaz 
        #opt value of v2
        vfull,_= Fullinfo_given_mu(mu,sigma,p2)
        #print([mu,vfull])
        v2= min(v1, vfull)
        
        #revenue if v1 and v2(mu) are known
        #revtemp= (p1-p2)*(scipy.special.erfc((v1-mu)/(math.sqrt(2)*sigma))/2) \
        revtemp = (p2-cost)*(scipy.special.erfc((v2-mu)/(math.sqrt(2)*sigma))/2)
        revsamples[s] = revtemp + firststage
        v2thresholds[s] = v2
    revstage= revsamples.mean()
    return revstage,revsamples,v1,v2thresholds

In [ ]:
def optrevNoinfo(pset,sigmaz,muz,sigma,cost):
    revno=np.zeros(len(pset))
    thresholds = np.zeros(len(pset))
    for idxi,p1 in enumerate(pset):
        noinfo_rev_temp,noinfo_threshold_temp,noinfo_adoption_temp = Noinfo(sigmaz,muz,sigma,p1,cost)
        revno[idxi] = noinfo_rev_temp
        thresholds[idxi] = noinfo_threshold_temp
    return revno.max(),pset[revno.argmax()],thresholds[revno.argmax()],thresholds

In [ ]:
def optrevFullinfo(pset,sigmaz,muz,sigma,cost,nsampling1stage,stdnormal_presampled):
    revfull=np.zeros(len(pset))
    thresholds = np.zeros((len(pset),nsampling1stage))
    for idxi,p2 in enumerate(pset):
        fulltemp,fulltemp_thresholds=Fullinfo(sigmaz,muz,sigma,p2,cost,nsampling1stage,stdnormal_presampled)
        revfull[idxi] = fulltemp
        thresholds[idxi,:] = fulltemp_thresholds
    return revfull.max(),pset[revfull.argmax()],thresholds[revfull.argmax(),:]

In [ ]:
def optrev2stage(pset,sigmaz,nuz,sigma,cost,nsampling2stage,stdnormal_presampled,noinfo_thresholds=None):
    starttime = time.time()
    
    if noinfo_thresholds is None:
        _,_,_,noinfo_thresholds = optrevNoinfo(pset,sigmaz,muz,sigtemp,cost)
    
    pset2additional = np.linspace(pset[-1],noinfo_thresholds.max(),num=100)
    pset2 = np.concatenate([pset,pset2additional])
    rev2stage = np.zeros((len(pset),len(pset2)))
    v12stage = np.zeros((len(pset),len(pset2)))
    v22stage = {}
    for idxi,p1 in enumerate(pset):
        if idxi%int(len(pset)/10)==0:
            print('sigma: ',sigma,'time elapsed: ',np.round(time.time()-starttime,3),' iter:',idxi,' p1:',p1)
        for idxj,p2 in enumerate(pset2):
            if p2 > p1:
                #v1set  = np.linspace(p1,pset[-1],num=10*int(pset[-1]-p1))
                noinfo_rev_temp,noinfo_treshold_temp,noinfo_adoption_temp = Noinfo(sigmaz,muz,sigma,p1,cost)
                if p2 >= noinfo_treshold_temp:
                    revtemp = noinfo_rev_temp
                    v1temp = noinfo_treshold_temp
                    v2thresholdstemp = np.inf*np.ones(nsampling2stage)
                else:
                    v1set  = np.linspace(p1,p1+2*sigma,num=int(2*sigma))
                    revtemp,_,v1temp,v2thresholdstemp = Rev2stage(sigmaz,muz,sigma,p1,p2,v1set,cost,nsampling2stage,stdnormal_presampled)
                rev2stage[idxi,idxj] = revtemp
                v12stage[idxi,idxj] = v1temp
                v22stage[(idxi,idxj)] = v2thresholdstemp
                    
    idxiopt, idxjopt = np.unravel_index(rev2stage.argmax(), rev2stage.shape)
    return rev2stage.max(),pset[idxiopt],pset2[idxjopt],v12stage[idxiopt,idxjopt],v22stage[(idxiopt,idxjopt)]

In [ ]:
def get_revs_full(pset,sigmaz,muz,cost,nsampling1stage,stdnormal_presampled,rev_curves,sigtemp):
    rn,rf,r2 = -np.inf,-np.inf,-np.inf
    prn,prf,pr2p1,pr2p2 = -np.inf,-np.inf,-np.inf,-np.inf
    th_no_vs_p1 = None
    th_no,th_full_nsampling1stage,th2_v1,th2_v2_nsampling2stage = None,None,None,None
    if 'no' in rev_curves:
        rn,prn,th_no,th_no_vs_p1 = optrevNoinfo(pset,sigmaz,muz,sigtemp,cost)
    if 'full' in rev_curves:
        rf,prf,th_full_nsampling1stage = optrevFullinfo(pset,sigmaz,muz,sigtemp,cost,nsampling1stage,stdnormal_presampled)
    if '2stg' in rev_curves:
        r2,pr2p1,pr2p2,th2_v1,th2_v2_nsampling2stage = \
        optrev2stage(pset,sigmaz,muz,sigtemp,cost,nsampling2stage,stdnormal_presampled,noinfo_thresholds=th_no_vs_p1)

    return {'sigma':sigtemp,'no':rn,'full':rf,'2stg':r2,'prn':prn,'prf':prf,'pr2':(pr2p1,pr2p2),
           'th_no':th_no,'th_full_nsampling1stage':th_full_nsampling1stage,'th2_v1':th2_v1,
           'th2_v2_nsampling2stage':th2_v2_nsampling2stage}

In [ ]:
#Experiment run

nsampling2stage = nsampling1stage
stdnormal_presampled = np.random.normal(0,1,max(nsampling1stage,nsampling2stage))
fname_results = 'results_muz'+str(muz)+'_sigmaz'+str(sigmaz)+'_cost'+str(cost)\
                +'_psetres'+str(len(pset))+'_sigmasetres'+str(len(sigmaset))+'_ncurves'+str(len(rev_curves))+'.pkl'

get_revs = partial(get_revs_full,pset,sigmaz,muz,cost,nsampling1stage,stdnormal_presampled,rev_curves)

print(len(pset))
start_time = time.time()
print('START TIME',time.ctime(start_time))
pool = mp.Pool(processes=mp.cpu_count())
results = pool.map(get_revs,list(sigmaset))
pickle.dump(results,open(fname_results,'wb'))
print('END TIME',time.ctime(time.time()), '. Time elapsed:',time.time() - start_time)

In [ ]:
#Plotting
results_temp = {x['sigma']:x for x in results}
revx_vs_sigma = np.zeros((3,len(sigmaset)))
for idxi,sigtemp in enumerate(sigmaset):
    revx_vs_sigma[0,idxi] = results_temp[sigtemp]['no']
    revx_vs_sigma[1,idxi] = results_temp[sigtemp]['full']
    revx_vs_sigma[2,idxi] = results_temp[sigtemp]['2stg']
fig, ax = plt.subplots(1, 1)
if len(rev_curves)==3:
    names = {0: 'Uninformed', 1:'Informed',2:'Two-stage'}
else:
    names = {0: 'Uninformed', 1:'Informed'}
for g in range(len(rev_curves)):
   ax.plot(sigmaset,revx_vs_sigma[g],label=names[g])
plt.legend(loc='best')
# plt.savefig(filename="tempplot.png")
plt.show()

In [ ]:
px_vs_sigma = np.zeros((4,len(sigmaset)))
for idxi,sigtemp in enumerate(sigmaset):
    px_vs_sigma[0,idxi] = results_temp[sigtemp]['prn']
    px_vs_sigma[1,idxi] = results_temp[sigtemp]['prf']
    px_vs_sigma[2,idxi] = results_temp[sigtemp]['pr2'][0]
    px_vs_sigma[3,idxi] = results_temp[sigtemp]['pr2'][1]
fig, ax = plt.subplots(1, 1)
if len(rev_curves)==2:
    names = {0: 'Uninformed Price', 1:'Informed Price'}
else:
    names = {0: 'Uninformed Price', 1:'Informed Price',2:'Two-stage Price 1',3:'Two-stage Price 2'}
for g in range(len(names)):
   ax.plot(sigmaset,px_vs_sigma[g],label=names[g])
plt.legend(loc='best')
plt.show()